<a href="https://colab.research.google.com/github/bozorgpanah/WASP-Software-Engineering-Project/blob/bozorgpanah-patch-1/Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Regression>>>predict the output of a continuous value
#Dataset>>>classic Auto MPG 
!pip install tensorflow --upgrade

# Use seaborn for pairplot
!pip install seaborn

# Use some functions from tensorflow_docs
!pip install git+https://github.com/tensorflow/docs

from __future__ import absolute_import, division, print_function, unicode_literals
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

print(tf.__version__)
name_db = "auto-mpg.data"
address_db = "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight', 'Acceleration', 'Model Year', 'Origin']
EPOCHS = 1000



#--downloading_dataset(name_db, address_db)

#Importing the data using Pandas library
def preprocessing_dataset(columnNames):
 raw_dataset = pd.read_csv(dataset_path, names=columnNames, na_values = "?", comment='\t', sep=" ", skipinitialspace=True)
 dataset = raw_dataset.copy() 
 dataset.tail()
 #Cleanin the data
 dataset.isna().sum()
 dataset = dataset.dropna()
 dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
 dataset = pd.get_dummies(dataset, prefix='', prefix_sep='') 
 return dataset.tail()

#Normalizing the data
def norm(x): 
  return (x - train_stats['mean']) / train_stats['std']

#Inspecting the model
def inspect_model():
 model = build_model()
 model.summary()
 example_batch = normed_train_data[:10]
 example_result = model.predict(example_batch)
 return example_result

#Making prediction
def prediction():
 test_predictions = model.predict(normed_test_data).flatten()

 a = plt.axes(aspect='equal')
 plt.scatter(test_labels, test_predictions)
 plt.xlabel('True Values [MPG]')
 plt.ylabel('Predictions [MPG]')
 lims = [0, 50]
 plt.xlim(lims)
 plt.ylim(lims)
 x_ = plt.plot(lims, lims)

 error = test_predictions - test_labels
 plt.hist(error, bins = 25)
 plt.xlabel("Prediction Error [MPG]")
 x_ = plt.ylabel("Count")  
 return plt.show()

 
#--preprocessing_dataset(columnNames)
#--split_dataset()
#--normed_train_data = norm(train_dataset) 
#--normed_test_data = norm(test_dataset)
#--build_model()
#--inspect_model()
#--train_model(EPOCHS)
#--prediction()


